# Expenses by Program and Category from 2012 - 2020

Expense data can be found on this site: http://otcads.umd.edu/bfa/budgetinfo3.htm  
For every "FY" year, there is a "Expenses - Program" and a "Expenses - Catageory" dataset. Basically, they're describing the expenses made that year and splitting it by program and category. Some of the data was available on a ".htm" file, so it was easy to scrape and return a pandas dataframe. A lot of the data was on pdf's, so each pdf had to be manually copied and pasted into strings. I then did some split and regex operations to separate out the data.

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import pickle

## Scraping & Parsing

For the years 2016, 2017, 2019, and 2020, there were ".htm" files. For each of these years, there were two html files that had to be parsed. One for the expenses by program and one for expenses by category. Therefore there were two unique url's for each year. Specifying the group (either 'Program' or 'Category') and the year made a unique url. 

In [2]:
def create_expense_df(url, group, year):
    r = requests.get(url)
    soup = BeautifulSoup(r.text,'html.parser')
    
    str_table = []

    table = soup.find_all('table')[0]
    for e in table.find_all('td'):
        if(not e.text.strip().isspace() and len(e.text.strip()) != 0):
            str_table.append(e.text.strip().replace('\r', '').replace('\n','').replace('\xa0', ''))
            
    i = str_table.index('AMOUNT') + 1

    expenses_tmp = pd.DataFrame(columns=['Year', 'Group Type', 'Group', 'Total Expenditure Amount'])

    while(i < len(str_table)):    
        dictionary = {
            'Year' : str(year),
            'Group Type' : group,
            'Group' : str_table[i].title(),
            'Total Expenditure Amount' : float(str_table[i + 1].replace(',', '').replace('$', ''))
        }
        expenses_tmp = expenses_tmp.append(dictionary, ignore_index=True)
        i = i + 2
    
    expenses_tmp['Year'] = expenses_tmp['Year'].astype(int)
    
    return expenses_tmp

Each year's expenses were appended to a list.

In [3]:
expenses_df = []
#  valid years 2016, 2017, 2019, 2020

# 2016
url1 = 'http://otcads.umd.edu/bfa/FY16%20Working%20Budget/web2/FY16%20EXP%20-%20PROG_files/sheet001.htm'
url2 = 'http://otcads.umd.edu/bfa/FY16%20Working%20Budget/web2/FY16%20EXP%20%20-%20CAT_files/sheet001.htm'
expenses_df.append(create_expense_df(url1, 'Program', 2016))
expenses_df.append(create_expense_df(url2, 'Category', 2016))

# 2017
url1 = 'http://otcads.umd.edu/bfa/FY17%20Working%20Budget/Web/FY17%20EXP%20%20-%20Prog%20web_files/sheet001.htm'
url2 = 'http://otcads.umd.edu/bfa/FY17%20Working%20Budget/Web/FY17%20EXP%20%20-%20CAT_files/sheet001.htm'
expenses_df.append(create_expense_df(url1, 'Program', 2017))
expenses_df.append(create_expense_df(url2, 'Category', 2017))

# 2019
url1 = 'http://otcads.umd.edu/bfa/FY19%20Working%20Budget/Web/FY19%20EXP%20%20-%20CAT%20Web%20by%20Prog%20revised_files/sheet001.htm'
url2 = 'http://otcads.umd.edu/bfa/FY19%20Working%20Budget/Web/FY19%20EXP%20%20-%20Web%20by%20CAT%20revised_files/sheet001.htm'
expenses_df.append(create_expense_df(url1, 'Program', 2019))
expenses_df.append(create_expense_df(url2, 'Category', 2019))

# 2020
url1 = 'http://otcads.umd.edu/bfa/FY20%20Working%20Budget/web/FY20%20EXP%20%20-%20Prog_files/sheet001.htm'
url2 = 'http://otcads.umd.edu/bfa/FY20%20Working%20Budget/web/FY20%20EXP%20%20-%20CAT_files/sheet001.htm'
expenses_df.append(create_expense_df(url1, 'Program', 2020))
expenses_df.append(create_expense_df(url2, 'Category', 2020))

## Copying pdf text and parsing this data into pandas dataframes

valid_scraping_years contains all the years for which url's can be scrapped.  
invalid_scraping_years contains all the years for which url's cannot be scraped and had to be manually copied and pasted below.

In [4]:
valid_scraping_years = set([2016, 2017, 2019, 2020])
invalid_scraping_years = set(list(range(2012, 2020 + 1))) - valid_scraping_years
invalid_scraping_years = sorted(invalid_scraping_years)
invalid_scraping_years

[2012, 2013, 2014, 2015, 2018]

`p_<year>` stands for expenses by program for that year  
`c_<year>` stands for expenses by category for that year  

In [5]:
p_2012 = """BY PROGRAM: AMOUNT
INSTRUCTION $428,495,207
RESEARCH 404,917,793
PUBLIC SERVICE 92,441,966
ACADEMIC SUPPORT 136,847,833
STUDENT SERVICES 43,124,048
INSTITUTIONAL SUPPORT 95,763,104
PLANT OPERATIONS 155,609,504
AUXILIARY ENTERPRISES 236,629,383
SCHOLARSHIPS & FELLOWSHIPS 94,130,248
TOTAL EXPENDITURES $1,687,959,086"""
c_2012 = """BY CATEGORY: AMOUNT
SALARIES & FRINGES $1,047,955,030
GRANTS (SCHOLARSHIPS & REMISSIONS) 128,602,978
FUEL & UTILITIES 69,648,583
EQUIPMENT & SUPPLIES & VEHICLE OPERATION 116,724,401
FIXED CHARGES, DEBT SERVICE, RENTALS & CONTRACTS 231,460,275
LAND & STRUCTURES 47,893,728
TRAVEL & COMMUNICATION 45,674,091
TOTAL EXPENDITURES $1,687,959,086"""

p_2013 = """BY PROGRAM: AMOUNT
INSTRUCTION $446,054,815
RESEARCH 408,760,233
PUBLIC SERVICE 110,955,322
ACADEMIC SUPPORT 134,705,448
STUDENT SERVICES 48,063,214
INSTITUTIONAL SUPPORT 102,279,771
PLANT OPERATIONS 155,881,837
AUXILIARY ENTERPRISES 237,091,682
SCHOLARSHIPS & FELLOWSHIPS 92,072,421
TOTAL EXPENDITURES $1,735,864,743"""
c_2013 = """BY CATEGORY: AMOUNT
SALARIES & FRINGES $1,065,834,126
GRANTS (SCHOLARSHIPS & REMISSIONS) 127,666,293
FUEL & UTILITIES 65,843,429
EQUIPMENT & SUPPLIES & VEHICLE OPERATION 117,275,986
FIXED CHARGES, DEBT SERVICE, RENTALS & CONTRACTS 253,546,074
LAND & STRUCTURES 56,688,108
TRAVEL & COMMUNICATION 49,010,727
TOTAL EXPENDITURES $1,735,864,743"""

p_2014 = """BY PROGRAM: AMOUNT
INSTRUCTION $462,203,052
RESEARCH 439,570,120
PUBLIC SERVICE 99,839,121
ACADEMIC SUPPORT 148,418,447
STUDENT SERVICES 48,874,791
INSTITUTIONAL SUPPORT 107,940,005
PLANT OPERATIONS 160,987,117
AUXILIARY ENTERPRISES 243,018,974
SCHOLARSHIPS & FELLOWSHIPS 101,483,978
TOTAL EXPENDITURES $1,812,335,605"""
c_2014 = """BY CATEGORY: AMOUNT
SALARIES & FRINGES $1,120,737,920
GRANTS (SCHOLARSHIPS & REMISSIONS) 133,482,141
FUEL & UTILITIES 62,014,508
EQUIPMENT & SUPPLIES & VEHICLE OPERATION 122,782,223
FIXED CHARGES, DEBT SERVICE, RENTALS & CONTRACTS 262,578,843
LAND & STRUCTURES 61,889,829
TRAVEL & COMMUNICATION 48,850,141
TOTAL EXPENDITURES $1,812,335,605"""

p_2015 = """BY PROGRAM: AMOUNT
INSTRUCTION $480,246,081
RESEARCH 427,338,193
PUBLIC SERVICE 93,187,167
ACADEMIC SUPPORT 158,885,956
STUDENT SERVICES 53,688,537
INSTITUTIONAL SUPPORT 108,205,546
PLANT OPERATIONS 163,714,975
AUXILIARY ENTERPRISES 263,330,275
SCHOLARSHIPS & FELLOWSHIPS 112,783,158
TOTAL EXPENDITURES $1,861,379,888"""
c_2015 = """BY CATEGORY: AMOUNT
SALARIES & FRINGES $1,169,565,756
GRANTS (SCHOLARSHIPS & REMISSIONS) 147,222,843
FUEL & UTILITIES 61,359,300
EQUIPMENT & SUPPLIES & VEHICLE OPERATION 122,558,188
FIXED CHARGES, DEBT SERVICE, RENTALS & CONTRACTS 238,077,554
LAND & STRUCTURES 71,589,344
TRAVEL & COMMUNICATION 51,006,903
TOTAL EXPENDITURES $1,861,379,888"""

p_2018 = """BY PROGRAM: AMOUNT
INSTRUCTION $555,109,091
RESEARCH 464,627,904
PUBLIC SERVICE 95,744,211
ACADEMIC SUPPORT 181,268,450
STUDENT SERVICES 60,056,385
INSTITUTIONAL SUPPORT 143,438,901
PLANT OPERATIONS 182,714,408
AUXILIARY ENTERPRISES 291,173,940
SCHOLARSHIPS & FELLOWSHIPS 116,234,379
TOTAL EXPENDITURES $2,090,367,669"""
c_2018 = """BY CATEGORY: AMOUNT
SALARIES & FRINGES $1,336,690,046
GRANTS (SCHOLARSHIPS & REMISSIONS) 150,148,887
FUEL & UTILITIES 61,057,636
EQUIPMENT & SUPPLIES & VEHICLE OPERATION 145,473,817
FIXED CHARGES, DEBT SERVICE, RENTALS & CONTRACTS 254,022,035
LAND & STRUCTURES 81,478,199
TRAVEL & COMMUNICATION 61,497,049
TOTAL EXPENDITURES $2,090,367,669"""

str_data = [p_2012, c_2012, p_2013, c_2013, p_2014, c_2014, p_2015, c_2015, p_2018, c_2018]

## Parsing

Now that all the data for the expenses has been painfully collected, the following code will parse the text and append expense dataframes to the expenses_df list.

In [6]:
expenses_tmp = pd.DataFrame()

for i in range(len(invalid_scraping_years)):
    
    lines = str_data[i].split('\n')
    year = int(invalid_scraping_years[i])
    
    group = ''
    for line in lines:
        if('BY PROGRAM: AMOUNT' in line):
            group = 'Program'
            continue
        if('BY CATEGORY: AMOUNT' in line):
            group = 'Category'
            continue
        
        elements = line.split()
        row_desc = ''
        for e in elements[:-1]:
            row_desc = row_desc + " " + e
        row_desc = row_desc.title()

        dictionary = {
            'Year' : int(year),
            'Group Type' : group,
            'Group' : row_desc,
            'Total Expenditure Amount' : float(elements[-1].replace(',', '').replace('$', ''))
        }
        expenses_tmp = expenses_tmp.append(dictionary, ignore_index=True)
        
expenses_df.append(expenses_tmp)

## Finalizing & Fine-tuning

Now the data is all combined into a single expense dataframe and relevant columns are correctly type casted.

In [7]:
expense = pd.concat(expenses_df, sort=False)
expense = expense[['Year', 'Group Type', 'Group', 'Total Expenditure Amount']]

In [8]:
expense['Total Expenditure Amount'] = expense['Total Expenditure Amount'].astype(float)
expense['Year'] = expense['Year'].astype(int)

In [9]:
expense = expense.sort_values(by='Year', ascending=False)

In [10]:
expense['Year'].unique()

array([2020, 2019, 2018, 2017, 2016, 2015, 2014, 2013, 2012])

In [11]:
expense

,Year,Group Type,Group,Total Expenditure Amount
5,2020,Program,Institutional Support,1.395546e+08
4,2020,Category,"Fixed Charges, Debt Service, Rentals & Contracts",2.754100e+08
0,2020,Program,Instruction,5.988685e+08
1,2020,Program,Research,4.956594e+08
2,2020,Program,Public Service,9.758716e+07
...,...,...,...,...
8,2012,Program,Scholarships & Fellowships,9.413025e+07
9,2012,Program,Total Expenditures,1.687959e+09
1,2012,Program,Research,4.049178e+08
0,2012,Program,Instruction,4.284952e+08


In [12]:
expense.to_pickle('df/expense')